In [1]:
# default_exp dictionary

# Dictionary Page Generation

<br>

### Imports

In [5]:
#exports
import json
import numpy as np
import pandas as pd

from jinja2 import Template

In [3]:
from IPython.display import JSON, Markdown

In [14]:
df_ids = pd.read_csv('../data/dictionary/ids.csv').set_index('dictionary_id')

for dict_id, dict_data in df_ids.iterrows():
    pass

dict_data.dropna().to_dict()

{'name': 'Winfrith', 'iaea_id': '239'}

In [5]:
#exports
def construct_contributor_txt(package):
    contributors = []
    
    for contributor in package.contributors:
        if 'title' in contributor.keys():
            title_key = 'title'
        elif 'name' in contributor.keys():
            title_key = 'name'
        else:
            raise ValueError(f'One of `title` or `name` must be provided with each licence, instead was passed: {", ".join(license.keys())}')
            
        if 'role' in contributor:
            contributors += [f'{contributor[title_key]} ({contributor["role"].capitalize()})']
        else:
            contributors += [contributor[title_key]]
    
    contributors_txt = ', '.join(contributors)
    
    return contributors_txt

def construct_licence_txt(package):
    licenses = []
    
    for license in package.licenses:
        assert 'name' in license, 'Each licence must have an associated `name`'
        
        if 'path' in license:
            licenses += [f'[{license["name"]}]({license["path"]})']
        else:
            licenses += [f'{license["name"]}']
    
    licence_txt = ', '.join(licenses)
    
    return licence_txt

def construct_metadata_table_str(package):
    attr_to_processing_func = {
        'version': lambda package: package.version,
        'contributors': lambda package: construct_contributor_txt(package),
        'keywords': lambda package: ', '.join(package.keywords),
        'licenses': lambda package: construct_licence_txt(package),
    }
    
    s_metadata = pd.Series({
        attr.capitalize(): attr_to_processing_func[attr](package)
        for attr
        in package.keys()
        if attr in attr_to_processing_func.keys()
    })
    
    s_metadata.index.name = 'Attribute'
    s_metadata.name = 'Value(s)'
    
    if s_metadata.size > 0:
        return s_metadata.to_markdown()
    else:
        return False

In [6]:
metadata_table_str = construct_metadata_table_str(package)

Markdown(metadata_table_str)

| Attribute    | Value(s)                                                  |
|:-------------|:----------------------------------------------------------|
| Keywords     | power plants, ids                                         |
| Version      | 1.0.0                                                     |
| Contributors | Ayrton Bourn (Author)                                     |
| Licenses     | [CC-BY-4.0](https://creativecommons.org/licenses/by/4.0/) |

In [7]:
#exports
def construct_field_desc_table_str(package, resource='ids'):
    s_field_descs = pd.Series({field['title']: field['description'] for field in package.get_resource(resource).schema.fields})

    s_field_descs.index.name = 'Field'
    s_field_descs.name = 'Description'
    
    md_str = s_field_descs.to_markdown()
    
    return md_str

In [8]:
field_desc_table_str = construct_field_desc_table_str(package)

Markdown(field_desc_table_str)

| Field                | Description                                                                                                                                                                                                                                        |
|:---------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Dictionary ID        | The identifier used to refer to power plants in the Subak data cooperative                                                                                                                                                                         |
| GPPD ID              | The identifier used in the global power plant database                                                                                                                                                                                             |
| ESAIL ID             | The identifier used by the Energy Systems and Artificial Intelligence Lab at UCL                                                                                                                                                                   |
| Common Name          | The name used to refer to the power plant site                                                                                                                                                                                                     |
| Settlement BMU ID    | The Balancing Mechanism Unit identifier used for settlement purposes by Elexon                                                                                                                                                                     |
| National Grid BMU ID | The Balancing Mechanism Unit identifier used by the National Grid                                                                                                                                                                                  |
| 4C-Offshore ID       | The identifier used by 4C-Offshore                                                                                                                                                                                                                 |
| WindPowerNet ID      | The identifier used by the WindPower.Net                                                                                                                                                                                                           |
| Wikidata ID          | The identifier used by Wikidata                                                                                                                                                                                                                    |
| Wikipedia ID         | The identifier used by Wikipedia                                                                                                                                                                                                                   |
| Power-Technology ID  | The identifier used by Power-Technology                                                                                                                                                                                                            |
| EUTL ID              | The identifier used in the European (Emissions Trading Scheme) Transaction Logs                                                                                                                                                                    |
| EIC ID               | The Energy Identification Code or EIC is a 16-character code used in Europe to uniquely identify entities and objects related to the electricity and gas sector. The EIC code is used for: Transmission System Operators, Market Participants etc. |
| CfD ID               | The identifier used by the Low Carbon Contracts Company to uniquely identify Contracts for Difference projects                                                                                                                                     |
| JRC ID               | Unique identifier of the hydro-power plant in the JRC data-base                                                                                                                                                                                    |
| IAEA ID              | Unique identifier used by the International Atomic Energy Agency                                                                                                                                                                                   |
| REPD ID (Old)        | Unique identifier used by BEIS in the Renewable Energy Planning Database - now superseded by a new ID scheme                                                                                                                                       |
| REPD ID (New)        | Unique identifier used by BEIS in the Renewable Energy Planning Database                                                                                                                                                                           |

In [26]:
#exports
def get_dp_field_to_url_format_str(datapackage_json_fp):
    package = Package(datapackage_json_fp, profile='tabular-data-package')
    ids_resource = package.get_resource('ids')

    id_field_to_url_format_str = {
        field['name']: field['url_format'] 
        for field 
        in ids_resource['schema']['fields'] 
        if 'url_format' in field.keys()
    }

    return id_field_to_url_format_str

def get_dp_field_to_title(datapackage_json_fp):
    package = Package(datapackage_json_fp, profile='tabular-data-package')
    ids_resource = package.get_resource('ids')

    id_field_to_title = {
        field['name']: field['title'] 
        for field 
        in ids_resource['schema']['fields'] 
    }

    return id_field_to_title

def format_url_ids(id_type, id_values, id_field_to_url_format_str, output_format='html'):
    if (id_type in id_field_to_url_format_str.keys()) and (id_values is not None):
        if output_format == 'markdown':
            return f'[{id_values}]({id_field_to_url_format_str[id_type].format(value=id_values)})' 
        elif output_format == 'html':
            return f'<a href="{id_field_to_url_format_str[id_type].format(value=id_values)}">{id_values}</a>' 
        else:
            raise ValueError(f'`output_format` must be one of: `html` or `markdown`')

    else:
        return id_values

def format_id_values(id_values, id_type, id_field_to_url_format_str, output_format='html'):
    if id_type in id_field_to_url_format_str.keys():
        id_values_strs = [format_url_ids(id_type, id_value, id_field_to_url_format_str, output_format) for id_value in id_values]
    else:
        id_values_strs = [str(id_value) for id_value in id_values] 
    
    return id_values_strs

def construct_linked_idxs(df_ids_clean, output_format='html', root_url='https://osuked.github.io/Power-Station-Dictionary/objects'):
    index = []
    
    for idx in df_ids_clean.index:
        if output_format == 'markdown':
            index += [f'[{idx}]({root_url}/{idx})']
        elif output_format == 'html':
            index += [f'<a href="{root_url}/{idx}">{idx}</a>']
        else:
            raise ValueError(f'`output_format` must be one of: `html` or `markdown`')
            
    return index

def construct_linked_ids_table_str(
    package, 
    datapackage_json_fp, 
    resource='ids', 
    output_format='html',
    output_kwargs={
        'index': False, 
        'table_id': 'dictionary'
    }
):            
    id_field_to_url_format_str = get_dp_field_to_url_format_str(datapackage_json_fp)
    id_field_to_title = get_dp_field_to_title(datapackage_json_fp)
    df_ids = package.get_resource(resource).to_pandas()
    
    df_ids_clean = pd.DataFrame(index=df_ids.index, columns=id_field_to_title.values())
    
    for dictionary_id, row in df_ids.iterrows():
        row = pd.Series({
            id_field_to_title[id_type]: (
                ', '.join([str(id_) for id_ in format_id_values(id_values, id_type, id_field_to_url_format_str, output_format)]) if isinstance(id_values, list) 
                else format_url_ids(id_type, id_values, id_field_to_url_format_str, output_format)
            ) 
            for id_type, id_values 
            in row.items()
        }).fillna('-')

        df_ids_clean.loc[dictionary_id] = row

    df_ids_clean = df_ids_clean.drop(columns='Dictionary ID')
    df_ids_clean.index = construct_linked_idxs(df_ids_clean, output_format)
    df_ids_clean.index.name = 'Dictionary ID'
    
    linked_ids_table_str = getattr(df_ids_clean.reset_index(), f'to_{output_format}')(**output_kwargs)
    
    linked_ids_table_str = linked_ids_table_str.replace('&lt;', '<')
    linked_ids_table_str = linked_ids_table_str.replace('&gt;', '>')
    linked_ids_table_str = linked_ids_table_str.replace('<td>', '<td class="truncate">')
    
    return linked_ids_table_str

In [27]:
linked_ids_table_str = construct_linked_ids_table_str(package, datapackage_json_fp, output_format='html')

# from IPython.display import HTML

# HTML(linked_ids_table_str)

In [28]:
#exports
def populate_and_save_template(template_fp, save_fp, **render_kwargs):
    rendered_str = Template(open(template_fp).read()).render(**render_kwargs)

    with open(save_fp, 'w', encoding='utf-8') as f:
        try:
            f.write(rendered_str)
        except e as exc:
            raise exc
            
    return None

def populate_dictionary_page(
    datapackage_json_fp: str='../data/dictionary/datapackage.json',
    template_fp: str='../templates/dictionary_page.md',
    save_fp: str=f'../docs/dictionary.md',
    output_format: str='html'
):
    package = Package(datapackage_json_fp, profile='tabular-data-package')

    render_kwargs = {
        'title': package.title,
        'description': package.description,
        'metadata_table': construct_metadata_table_str(package),
        'field_desc_table': construct_field_desc_table_str(package),
        'linked_ids_table': construct_linked_ids_table_str(package, datapackage_json_fp, output_format=output_format),
    }

    populate_and_save_template(template_fp, save_fp, **render_kwargs)
    
    return

In [29]:
populate_dictionary_page(
    datapackage_json_fp = '../data/dictionary/datapackage.json',
    template_fp = '../templates/dictionary_page.md',
    save_fp = f'../docs/dictionary.md'
)

In [30]:
#hide
from nbdev.export import *
notebook2script()

Converted 00-documentation.ipynb.
Converted 01-dictionary-page.ipynb.
Converted 02-attribute extraction.ipynb.
Converted 03-page-population.ipynb.
Converted 04-cli.ipynb.
Converted 05-carbon-intensity.ipynb.
Converted 06-cfd-capture-price-comparison.ipynb.
Converted 07-dataset-pages.ipynb.
Converted 08-papers.ipynb.
Converted 09-id-submission.ipynb.
